# Dummy Net

 - prediction only
 - check graph
 - loss function 
 - add loss summaries
 - tune optimizer
 - layer_2 with new shape, initialize with zeros
 - layer_2 with constant_initializer(0.1)
 - improve steps and lr
 - add saver

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import tensorflow as tf
import numpy as np
import os
import shutil
from utils.data import init_dir

## Build Net Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('dummy_net_inputs'):
        data = tf.placeholder(tf.float32, shape=[None, 3], name='data')
        labels = tf.placeholder(tf.float32, shape=[None, 1], name='labels')
    
    with tf.variable_scope('layer_1'):
        w = tf.get_variable('w', initializer=tf.constant_initializer(0), shape=[3,2])
        prediction = tf.matmul(data, w)
    
    with tf.variable_scope('layer_2'):
        w = tf.get_variable('w', initializer=tf.constant_initializer(0.1), shape=[2,1])
        prediction = tf.matmul(prediction, w)
    
    with tf.name_scope('loss'):
        loss = tf.losses.mean_squared_error(labels, prediction)
        tf.summary.scalar('loss_summary', loss)

    with tf.name_scope('training'):
        train_step = tf.train.GradientDescentOptimizer(0.005).minimize(loss)

    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

## Init Direcotory Structure

In [ ]:
base_dir = '/tensorboard_summaries/dummy_net/'

#### Init Experiment Dir

In [ ]:
exp_name = 'experiment_1'
exp_dir = init_dir(base_dir, exp_name, remove_existing=True)

#### Init Checkpoint Path

In [ ]:
model_path = os.path.join(exp_dir, 'model.ckpt')

## Init Saver and Logger

In [ ]:
with graph.as_default():
    saver = tf.train.Saver()
file_writer = tf.summary.FileWriter(exp_dir, graph)

## Run net

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True

_data = np.array([[1,2,3]])
_labels = np.array([[5]])

with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    #saver.restore(session, model_path)
    
    for iteration in range(1000):
        feed_dict = {
            data: _data, 
            labels: _labels
        }
        fetches = [train_step, prediction, loss, merge_summaries]
        
        _, _prediction, _loss, _summary = session.run(fetches, feed_dict)
        file_writer.add_summary(_summary, iteration)
        
        print("Iteration {}: loss {}, pred {}".format(iteration, _loss, _prediction[0][0]))
    saver.save(session, model_path)